In [112]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import time
import pandas as pd
import re

# 事前準備
url = 'https://suumo.jp/chintai/tokyo/sc_koto/?page={}'
d_list = []

for i in range(1,10):
    target_url = url.format(i)
    r = requests.get(target_url)
    
    # 1秒ずつ取得
    time.sleep(1)
    soup = BeautifulSoup(r.text,"html.parser")
    
    contents = soup.find_all('div', class_='cassetteitem')

    for content in contents:
        detail = content.find('div', class_='cassetteitem-detail')
        table = content.find('table', class_='cassetteitem_other')
        
        # タイトルを取得し、空白を削除
        title_text = detail.find('div', class_='cassetteitem_content-title').text
        title = title_text.replace('　', '')
        
        # アドレスを取得
        address = detail.find('li', class_='cassetteitem_detail-col1').text
        
        # アクセスを取得、改行を削除、要素間をカンマ区切りに
        access_text = detail.find('li', class_='cassetteitem_detail-col2').text
        cleaned_access_text = access_text.strip()
        access = cleaned_access_text.replace('\n', ',')
        
        # 築年数と総階数を取得、分割してageとstoryに格納。地下は重要度低いため除外
        spaced_age_and_story = detail.find('li', class_='cassetteitem_detail-col3').text
        age_and_story = spaced_age_and_story = spaced_age_and_story.replace('築0年', '築1年').replace('新築', '築1年')
        numbers = re.findall(r'\d+', age_and_story)
        if len(numbers) == 2:
            age = int(numbers[0])
            story = int(numbers[1])
        elif len(numbers) == 3:
            age = int(numbers[0])
            story = int(numbers[2])
        else:
            age = 0
            story = 0
            
        # trタグから部屋情報を取得
        trtags = table.find_all('tr', class_='js-cassette_link')
        for trtag in trtags:
            original_floor_data, price, first_fee, capacity = trtag.find_all('td')[2:6]
            
            # 階数を取得、複数階は最低階を格納、階なしは1を格納
            floor_text = original_floor_data.text
            matched_floor_number = re.search(r'\d+', floor_text)
            if matched_floor_number:
                floor = int(matched_floor_number.group())
            else:
                floor = 1
            
            # strを引数、数字のみ抽出したfloat型のnumberを返す関数
            def extract_number(text):
                matched_text = re.search(r'\d+(\.\d+)?', text)
                if matched_text:
                    number = float(matched_text.group())
                else:
                    number = 0
                return number
            
            # 各オブジェクトに関数適用してfloat型に揃える
            fee_text, management_fee_text = [li.text for li in price.find_all('li')]
            fee = extract_number(fee_text)
            management_fee = extract_number(management_fee_text)/10000 # 管理費も万円に揃える
            
            deposit_text, gratuity_text = [li.text for li in first_fee.find_all('li')]
            deposit = extract_number(deposit_text)
            gratuity = extract_number(gratuity_text)
            
            madori, menseki_text = [li.text for li in capacity.find_all('li')]
            menseki = extract_number(menseki_text)
            
            d = {
                'title': title,
                'address': address,
                'access': access,
                'age': age,
                'story': story,
                'floor': floor,
                'fee': fee,
                'management_fee': management_fee,
                'deposit': deposit,
                'gratuity': gratuity,
                'madori': madori,
                'menseki': menseki,
            }
            d_list.append(d)

df = pd.DataFrame(d_list)

In [114]:
df.head()

,title,address,access,age,story,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,プラウドシティ越中島ブライトテラス,東京都江東区越中島３,"ＪＲ京葉線/越中島駅 歩6分,東京メトロ東西線/門前仲町駅 歩12分,東京メトロ有楽町線/豊...",5,12,4,25.0,0.0,50.0,25.0,3SLDK,72.87
1,ルカナル門前仲町,東京都江東区富岡２,"東京メトロ東西線/門前仲町駅 歩7分,東京メトロ東西線/木場駅 歩6分,ＪＲ京葉線/越中島駅...",4,10,2,11.6,0.7,11.6,11.6,1DK,30.26
2,ルカナル門前仲町,東京都江東区富岡２,"東京メトロ東西線/門前仲町駅 歩7分,東京メトロ東西線/木場駅 歩6分,ＪＲ京葉線/越中島駅...",4,10,4,17.9,1.0,17.9,35.8,2LDK,50.73
3,東京メトロ東西線 木場駅 10階建 築4年,東京都江東区富岡２,"東京メトロ東西線/木場駅 歩6分,都営大江戸線/門前仲町駅 歩7分,ＪＲ京葉線/越中島駅 歩15分",4,10,2,11.6,0.7,11.6,11.6,1DK,30.26
4,東京メトロ東西線 木場駅 10階建 築4年,東京都江東区富岡２,"東京メトロ東西線/木場駅 歩6分,都営大江戸線/門前仲町駅 歩7分,ＪＲ京葉線/越中島駅 歩15分",4,10,4,17.9,1.0,17.9,35.8,2LDK,50.73
